<a href="https://www.quantrocket.com"><img alt="QuantRocket logo" src="https://www.quantrocket.com/assets/img/notebook-header-logo.png"></a>

<a href="https://www.quantrocket.com/disclaimer/">Disclaimer</a>

***
[Pipeline Tutorial](Introduction.ipynb) › Lesson 11: Custom Factors
***

# Custom Factors
When we first looked at factors, we explored the set of built-in factors. Frequently, a desired computation isn't included as a built-in factor. One of the most powerful features of the Pipeline API is that it allows us to define our own custom factors. When a desired computation doesn't exist as a built-in, we define a custom factor.

Conceptually, a custom factor is identical to a built-in factor. It accepts `inputs`, `window_length`, and `mask` as constructor arguments, and returns a `Factor` object each day.

Let's take an example of a computation that doesn't exist as a built-in: standard deviation. To create a factor that computes the [standard deviation](https://en.wikipedia.org/wiki/Standard_deviation) over a trailing window, we can subclass `zipline.pipeline.CustomFactor` and implement a compute method whose signature is:


```
def compute(self, today, asset_ids, out, *inputs):
    ...
```

- `*inputs` are M x N numpy arrays, where M is the `window_length` and N is the number of securities (usually around ~8000 unless a `mask` is provided). `*inputs` are trailing data windows. Note that there will be one M x N array for each `BoundColumn` provided in the factor's `inputs` list. The data type of each array will be the `dtype` of the corresponding `BoundColumn`.
- `out` is an empty array of length N. `out` will be the output of our custom factor each day. The job of the `compute` method is to write output values into `out`.
- `asset_ids` will be an integer array of length N containing security ids corresponding to the columns in our `*inputs` arrays.
- `today` will be a pandas Timestamp representing the day for which `compute` is being called.

Of these, `*inputs` and `out` are most commonly used.

An instance of `CustomFactor` that has been added to a pipeline will have its compute method called every day. For example, let's define a custom factor that computes the standard deviation of the close price over the last 5 days. To start, let's add `CustomFactor` and `numpy` to our import statements.

In [1]:
from zipline.pipeline import Pipeline, EquityPricing
from zipline.pipeline.factors import CustomFactor
from zipline.research import run_pipeline
import numpy

Next, let's define our custom factor to calculate the standard deviation over a trailing window using `numpy.nanstd`:

In [2]:
class StdDev(CustomFactor):
    def compute(self, today, asset_ids, out, values):
        # Calculates the column-wise standard deviation, ignoring NaNs
        out[:] = numpy.nanstd(values, axis=0)

Finally, let's instantiate our factor in `make_pipeline()`:

In [3]:
def make_pipeline():
    std_dev = StdDev(inputs=[EquityPricing.close], window_length=5)

    return Pipeline(
        columns={
            'std_dev': std_dev
        }
    )

When this pipeline is run, `StdDev.compute()` will be called every day with data as follows:

- `values`: An M x N numpy array, where M is 5 (`window_length`), and N is ~8000 (the number of securities in our database on the day in question).
- `out`: An empty array of length N (~8000). In this example, the job of `compute` is to populate `out` with an array storing the 5-day close price standard deviations.

In [4]:
result = run_pipeline(make_pipeline(), start_date='2015-05-05', end_date='2015-05-05')
result

/opt/conda/lib/python3.9/site-packages/numpy/lib/nanfunctions.py:1670: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


std_dev
date                      asset                                   
2015-05-05 00:00:00+00:00 Equity(FIBBG000C2V3D6 [A])      0.268224
                          Equity(QI000000004076 [AABA])   0.801359
                          Equity(FIBBG006T1NZ18 [AAC])    1.436195
                          Equity(FIBBG000V2S3P6 [AACG])   0.132151
                          Equity(FIBBG000BDYRW6 [AADR])   0.067500
...                                                            ...
                          Equity(FIBBG0001JCK35 [RBSPE])       NaN
                          Equity(FIBBG0001JCK44 [RBSPG])       NaN
                          Equity(FIBBG0001JCQT4 [RBSPI])       NaN
                          Equity(FIBBG007NFXFG8 [UZB])    0.081388
                          Equity(FIBBG0046M31Y3 [WMGIZ])  0.036818

[8292 rows x 1 columns]

### Default Inputs
When writing a custom factor, we can set default `inputs` and `window_length` in our `CustomFactor` subclass. For example, let's define the `TenDayMeanDifference` custom factor to compute the mean difference between two data columns over a trailing window using `numpy.nanmean`. Let's set the default `inputs` to `[EquityPricing.close, EquityPricing.open]` and the default `window_length` to 10:

In [5]:
class TenDayMeanDifference(CustomFactor):
    # Default inputs.
    inputs = [EquityPricing.close, EquityPricing.open]
    window_length = 10
    def compute(self, today, asset_ids, out, close, open):
        # Calculates the column-wise mean difference, ignoring NaNs
        out[:] = numpy.nanmean(close - open, axis=0)

<i>Remember in this case that `close` and `open` are each 10 x ~8000 2D numpy arrays.</i>

If we call `TenDayMeanDifference` without providing any arguments, it will use the defaults.

In [6]:
# Computes the 10-day mean difference between the daily open and close prices.
close_open_diff = TenDayMeanDifference()

The defaults can be manually overridden by specifying arguments in the constructor call.

In [7]:
# Computes the 10-day mean difference between the daily high and low prices.
high_low_diff = TenDayMeanDifference(inputs=[EquityPricing.high, EquityPricing.low])

### Further Example
Let's take another example where we build a [momentum](http://www.investopedia.com/terms/m/momentum.asp) custom factor and use it to create a filter. We will then use that filter as a `screen` for our pipeline.

Let's start by defining a `Momentum` factor to be the division of the most recent close price by the close price from `n` days ago where `n` is the `window_length`.

In [8]:
class Momentum(CustomFactor):
    # Default inputs
    inputs = [EquityPricing.close]

    # Compute momentum
    def compute(self, today, assets, out, close):
        out[:] = close[-1] / close[0]

Now, let's instantiate our `Momentum` factor (twice) to create a 10-day momentum factor and a 20-day momentum factor. Let's also create a `positive_momentum` filter returning `True` for securities with both a positive 10-day momentum and a positive 20-day momentum.

In [9]:
ten_day_momentum = Momentum(window_length=10)
twenty_day_momentum = Momentum(window_length=20)

positive_momentum = ((ten_day_momentum > 1) & (twenty_day_momentum > 1))

Next, let's add our momentum factors and our `positive_momentum` filter to `make_pipeline`. Let's also pass `positive_momentum` as a `screen` to our pipeline.

In [10]:
def make_pipeline():

    ten_day_momentum = Momentum(window_length=10)
    twenty_day_momentum = Momentum(window_length=20)

    positive_momentum = ((ten_day_momentum > 1) & (twenty_day_momentum > 1))

    std_dev = StdDev(inputs=[EquityPricing.close], window_length=5)

    return Pipeline(
        columns={
            'std_dev': std_dev,
            'ten_day_momentum': ten_day_momentum,
            'twenty_day_momentum': twenty_day_momentum
        },
        screen=positive_momentum
    )

Running this pipeline outputs the standard deviation and each of our momentum computations for securities with positive 10-day and 20-day momentum.

In [11]:
result = run_pipeline(make_pipeline(), start_date='2015-05-05', end_date='2015-05-05')
result

/opt/conda/lib/python3.9/site-packages/numpy/lib/nanfunctions.py:1670: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


std_dev  \
date                      asset                                     
2015-05-05 00:00:00+00:00 Equity(FIBBG006T1NZ18 [AAC])   1.436195   
                          Equity(FIBBG000BDYRW6 [AADR])  0.067500   
                          Equity(FIBBG000D9V7T4 [PRG])   0.282942   
                          Equity(FIBBG000B9XRY4 [AAPL])  1.770774   
                          Equity(FIBBG000C5QZ62 [AAV])   0.063056   
...                                                           ...   
                          Equity(FIBBG0019HMFX6 [ZIV])   0.398768   
                          Equity(FIBBG000H04C72 [ZIXI])  0.091345   
                          Equity(FIBBG005WX1JJ7 [ZMLP])  0.139857   
                          Equity(FIBBG000RFZLM7 [ZN])    0.032727   
                          Equity(FIBBG011MC2100 [AATC])  0.149987   

                                                         ten_day_momentum  \
date                      asset                                             
2015-05-05 00:00:00+00:00 Equity(FIBBG006T1NZ18 [AAC])           1.033223   
                          Equity(FIBBG000BDYRW6 [AADR])          1.004599   
                          Equity(FIBBG000D9V7T4 [PRG])           1.217940   
                          Equity(FIBBG000B9XRY4 [AAPL])          1.014104   
                          Equity(FIBBG000C5QZ62 [AAV])           1.004950   
...                                                                   ...   
                          Equity(FIBBG0019HMFX6 [ZIV])           1.016408   
                          Equity(FIBBG000H04C72 [ZIXI])          1.072639   
                          Equity(FIBBG005WX1JJ7 [ZMLP])          1.012349   
                          Equity(FIBBG000RFZLM7 [ZN])            1.053191   
                          Equity(FIBBG011MC2100 [AATC])          1.139194   

                                                         twenty_day_momentum  
date                      asset                                               
2015-05-05 00:00:00+00:00 Equity(FIBBG006T1NZ18 [AAC])              1.058478  
                          Equity(FIBBG000BDYRW6 [AADR])             1.027848  
                          Equity(FIBBG000D9V7T4 [PRG])              1.237892  
                          Equity(FIBBG000B9XRY4 [AAPL])             1.021348  
                          Equity(FIBBG000C5QZ62 [AAV])              1.074074  
...                                                                      ...  
                          Equity(FIBBG0019HMFX6 [ZIV])              1.054225  
                          Equity(FIBBG000H04C72 [ZIXI])             1.135897  
                          Equity(FIBBG005WX1JJ7 [ZMLP])             1.037026  
                          Equity(FIBBG000RFZLM7 [ZN])               1.076087  
                          Equity(FIBBG011MC2100 [AATC])             1.196154  

[2752 rows x 3 columns]

Custom factors allow us to define custom computations in a pipeline. They are frequently the best way to perform computations on multiple data columns. The full documentation for CustomFactors is available in the [API Reference](https://www.quantrocket.com/docs/api/#zipline.pipeline.CustomFactor).

---

**Next Lesson:** [Creating the TradableStocksUS Universe](Lesson12-TradableStocksUS-Universe.ipynb) 